# Tell


In the previous notebook, we worked with a general-purpose *mathematical* programming problem, the *knapsack problem*, which seeks to maximise the benefit gained from "packing" a set of objects such that all objects fit into the knapsack. We use binary decision variables $x_i$ to represent whether or not an item is taken (with $i=1...n$). Items each have some benefit $b_i$ and size $s_i$; our knapsack has a total size of $c$, past which we cannot pack any more items in. With these symbols, we state the *knapsack problem* as: 

<div class="alert alert-success" role="alert">
<b>The Knapsack Problem </b>


*Maximize:* 

$\sum_i^n b_ix_i$

*Subject To:*

1. *Size constraint:*&emsp; $\sum_i^n s_i x_i \leq c$
2. *Binary decisions:*&emsp; $x_i \in {0,1} \ \ \forall \ \ i = 1, 2, ..., n$

</div>

Spatial optimisation problems get slightly more complex than this, both in terms of the numbers and the types of constraints. 

## The unique challenge of spatial optimisation

Generally, any *spatial* optimisation problem requires us to use a *spatial decision variable* to represent the spatial *locations of* or *relation between* decisions we take. Often, *relation* decisions are either *allocation* or *routing* decisions, which will be explained in detail below. Spatial optimisation can be quite challenging because the number of decisions we have to make about spatial *relations* will balloon rapidly as the problem size gets larger.

Recall that the knapsack problem is an *aspatial* optimisation problem: items are either "packed" or they're "not packed." This "packed"/"not packed" decision variable is similar to a *locational* decision variable. When we are are trying to determine which site is the best to *locate* a facility, we use one binary *locational decision variable* for each item being located: is site A chosen for a facility or not? Note that this means there is no information about the *relation* of items in the knapsack or locations being sited to one another. We don't use a decision variable that indicates we would prefer to pack pencils *more than* packing pens, or locate at site A *instead of* site B, for example. This preference is implicit in the final set of decisions: do we pack pens *and* pencils, or just pack *pencils*? 

Many *other* kinds of spatial optimisation problems do require *relational decision variables* that represent decisions we make that relate two things. A very common example of a relational decision variable is an *allocation* variable that tracks whether we *allocate* an object to a given class. For example, an "allocation" decision in a knapsack example might be needed if you were trying to figure out whether to take chewing gum *in your backpack* or *in your pocket*. The "location-style" decision variable is "do I pack chewing gum?", and the "allocation-style" decision is that "do I pack the chewing gum in my pocket or in my backpack?" In fact, many spatial optimisation problems are *location-allocation* problems like this, where locational decisions about "do we locate a facility at site A?" are paired with *allocation* decisions: does the facility at site A serve client 1? Often, we intend to *allocate* some larger number of things (clients, items to take to school) to a smaller number of things (facilities, backpack or pocket). This *location-allocation* framework covers many of the most important problems in spatial optimisation. 

Another kind of *spatial* optimisation problems involves *routing* decisions. These are sometimes relatively easy, like solving for the [*shortest path*](https://en.wikipedia.org/wiki/Shortest_path_problem) from one location to another. But, other kinds of problems, like trying to find an optimal [*delivery route*](https://en.wikipedia.org/wiki/Travelling_salesman_problem), are quite difficult. In both of these problems, the *spatial decision variable* involves a choice about whether or not a *link* between two *waypoints* is selected or not. Similar to an *allocation* variable, the decision variable about the *route* is a binary variable for *each pair of waypoints*: do we go from site A to site B? 

While this sounds simple, the number of decisions being made grows quickly for *allocation* and *routing* problems. Generally, problems that involve only *location* decisions are the smallest and easier to solve, followed by those that involve *allocation*, and finally those that involve *routing*. And, *location* problems are already among the class of hardest computational problems to solve to optimality, so "easier" does not mean "easy"! Problems are made more complex when decisions are *contingent upon* other decisions, hence a *location-allocation* problem will be harder to solve generally than an *allocation* problem where the location decisions are already made. Problems that entail all three (location-allocation-routing) are quite difficult to solve indeed. 

Let us think about a concrete example to make this clearer. Imagine we have 8 delivery trucks split across 4 possible depot sites. But, we only have the funding to operate *two* depots. Each truck should start and end its delivery route in its depot. Over the course of the day, we need to deliver to 5,000 housing units in total. In this case, we can illustrate the three typical kinds of *spatial* decision variables as the following: 

1. *locating depots*: do we locate a depot at site $i$? This is often just a binary decision variable, $x_i$, for each possible depot site. Thus, it represents $4$ decision variables.
2. *allocating trucks to depots*: does truck $j$ get allocated to depot $i$? This is a small matrix of decision variables, $\mathbf{Y}$, with one $z_{ij}$ for each $i,j$ pair representing whether truck $j$ will be allocated to depot $i$. It represents $8 * 4$ decision variables. 
3. *allocating houses to trucks*: does house $k$ get allocated to truck $j$? This is a matrix of decision variables $\mathbf{Z}$, with one $Z_{jk}$ for each $j,k$ pair representing whether truck $j$ will be allocated to deliver to house $k$. It represents $8 * 5,000$ decision variables overall. 
4. *routing trucks*: what route does truck $j$ take to serve its allocated houses $k$? Classically, this is represented by a $k \times k$ matrix for *each* $j$, $\mathbf{R}_j$, that records whether the link between $k_1$ and $k_2$ is selected for the route. This results in $k^2$ decision variables *per truck*, which (optimistically, when all routes have the same size) is just over $3,000,000$ decision variables for all trucks! 

Thus, you can see how a simple problem like parcel delivery gets very large, very quickly. We will cover decisions 1 and 3 to discuss an introductory spatial optimisation problem. 

 ## Introducing classical location-allocation: the P-Median problem

The $p$-Median problem is the challenge of locating some set number ($p$) of "medians" so that they minimise the distance to a set of sites we're trying to service. For our purposes, we will also refer to the medians as "facilities" and the sites we're trying to service as "clients," to match the `spopt` package's terms. Sometimes, facilities can be located anywhere (in the "continuous P-Median" model). In our case, we will focus on cases where medians can only occupy fixed locations, the "discrete P-Median" model, which is more common in planning and site selection tasks. 

In this case, we have a collection of decisions to make. First, we make a locational decision: do we open a facility at site $j$? If we have $m$ possible sites for facilities, this can be represented with $m$ decision variables ($y_j$). Then, we need to *allocate* clients to open facilities. If we have $n$ clients, this requires us to build a "client-to-facility" matrix ($\mathbf{Z}$)of shape $n \times m$ to record the allocation decisions for each client to each facility, $z_{ij}$. Note that this allocation decision of site $i$ to facility $j$ is *contingent upon* the location decision at $j$: we cannot assign any clients to a "closed" facility ($y_j = 0$)! 

Finally, we need to model how "good" our problem solution is. Generally in median problems, we define the "cost" of an allocation to represent the effort required to satisfy the allocation: we consider the *distance* between each client and facility pair and *how much* has to be carried over that distance. This generally represents the "work" you do for that allocation decision, and is flexible enough to accommodate nonlinear functions of distance (e.g. increasing costs to travel with a squared distance or decreasing costs to travel with a logarithmic distance). For the clasical $p$-Median problem, we represent our "work" as the distance ($d_{ij}$) between facilities and their allocated clients *times* the amount of stuff we're carrying from facilty to client ($w_i$). But, we will *only* carry something from facility $j$ to client $i$ *if* we make an allocation decision $z_{ij}$. Thus, for all client-facility pairs, we want to minimise the sum of the weighted distances between clients and assigned facilities: 

$$ d_{11}z_{11}w_{1} + d_{12}z_{12}w_{1} + ... + d_{1m}z_{1m}w_{1} + d_{21}z_{21}w_2 + d_{22}z_{22}w_2 + ... + d_{nm}z_{nm}w_{n} = \sum_i^n\sum_j^m d_{ij}z_{ij}w_i$$

Thus, when client $i$ is assigned to facility $j$ ($z_{ij}=1$), we get $d_{ij}*1*w_i=d_{ij}w_i$, distance times load size. But, when client $i$ is *not* assigned to facility $j$, $z_{ij} = 0$, so $d_{ij}*0*w_i=0$. This is how we represent the sum of distances between clients and their allocated facility, the *objective* of the p-median model. 

To complete the model, we need to make sure that (1) only $p$ facilities are located and (2) every client is allocated to one open facility. For (1), we can use something like our size constraint from the knapsack problem: 

$$ \sum_{j}^{m} y_{j} = p$$

This *constraint* ensures that exactly $p$ facilities are located, since the sum of $y_i$ represents the number of facilities opened. For (2), we have to make sure that every client is assigned to one (and only one) facility. So, we have to write a constraint for *each client* to ensure it gets assigned to one facility. 
$$ \sum_{j}^{m} z_{ij} = 1$$

Thus, we will have $n$ constraints that ensure that clients are assigned to exactly one open facility. But, note that this constraint *does not* ensure that we allocate client $i$ to an _open_ facility $j$: $z_{ij}$ could be 1 while $y_j$ could be zero! Thus, we need an *allocation* constraint linking $z$ and $y$, to ensure that no client can be assigned to a closed facility. This requires a constraint on *each decision variable* $z_{ij}$ that ensures $z_{ij}$ can only be $1$ if $y_j$ is one: 

$$ z_{ij} \leq y_j $$

Thus, the siutation we describe above where a client is assigned to a closed facility (in math, $z_{ij} = 1$ when $y_{j} = 0$) can never happen *so long as* this constraint is used. 

Together, these comprise the P-Median problem: 

<div class="alert alert-success" role="alert">
<b>The P Median Problem</b>

*Minimise* 

$\sum_i^n\sum_j^m d_{ij}z_{ij}w_{i}$

*Subject To:*

1. *Locate $p$ Facilities*:&emsp;&emsp;&emsp;&emsp;&ensp;&nbsp; $ \sum_{j}^{m} y_{j} = p$
2. *Allocate Clients Once*:&emsp;&emsp;&emsp; $ \sum_{j}^{m} z_{ij} = 1 \ \ \ \ \forall i$
3. *Allocate to Open Facilities*:&emsp; $ z_{ij} \leq y_j \ \ \ \ \forall i,j$
4. *Binary Location Decisions*:&emsp; $y_j \in {0,1} \ \ \ \ \forall j$ <br>&emsp;&emsp;&emsp;&emsp;&ensp;&emsp;&emsp;&emsp;&emsp;&ensp;&emsp;&emsp;&emsp;&emsp;   $z_{ij} \in {0,1} \ \ \ \ \forall i,j$

</div>

You can see that this problem requires some information about the relationships between clients and facilities, $d_{ij}$, in order to *solve* for the values of decision variables $z_{ij}$ and $y_j$. Thus, we will walk you through how to set up this problem for yourself using the `spopt` package, along with a few other geospatial packages. 

Also a few caveats are in order. First, when the weight of each demand is constant, we can ignore $w_i$ in our formulation and solve a simpler *unweighted $p$-Median model*. However, this is rarely the case, so we'll consider the typical *weighted $p$-Median model*. Also, note that if we allocate client $i$ to facility $j$, we're committing to serving *all of* the demand client $i$ has using facility $j$; no other facility is allowed to help. If we relax this and allow for multiple facilities to help out with serving demand $i$, this is known as a *fractional allocation* $p$-Median model. This is encountered occasionally, but can be challenging to explain and investigate the results, so we will keep to the total allocation version. 

# Show

For this example, we will solve for parcel depots in inner-city Bristol. We will use a few more libraries this time: 

In [1]:
import pandas
import pulp
import geopandas # standard library for geographical data in Python
import numpy # standard library for numerical and array-based computing
import spopt # library for spatial optimisation

To implement a $p$-Median problem, we will start with data on the location of postcodes in the UK: 

In [2]:
postcodes = pandas.read_csv("./uk_postcodes.csv")
postcodes.head()

,outward,easting,northing,lat,lon,demand
0,AB1,383656,760468,56.735562,-2.268770,84.409506
1,AB10,392567,804537,57.131679,-2.124423,121.898496
2,AB11,394533,805406,57.139507,-2.091960,93.939568
3,AB12,393057,800339,57.093971,-2.116218,115.097898
4,AB13,387085,802538,57.113599,-2.214879,57.817527


Further, since we have geographic data (in terms of the latitude and longitude of postcode centroids), we will use `geopandas` to make maps and work with the spatial data. 

In [3]:
postcodes = geopandas.GeoDataFrame(
    postcodes, 
    geometry=geopandas.points_from_xy(postcodes.lon, postcodes.lat, crs="epsg:4326")
)

To keep the analysis tractable, we will also focus *only* on inner-city Bristol postcodes: 

In [4]:
bristol_postcodes = postcodes[postcodes.outward.str.startswith("BS")].copy()

# is the postcode in the center of the city? 
is_inner_city = bristol_postcodes.outward.str.lstrip("BS").astype(int) < 10

# if so, keep it around
bristol_innercity = bristol_postcodes[is_inner_city].copy()

To visualise, this results in only nine postcodes in the center of the city: 

In [5]:
bristol_innercity

,outward,easting,northing,lat,lon,demand,geometry
264,BS1,358981,174319,51.466347,-2.591879,119.066167,POINT (-2.59188 51.46635)
275,BS2,359125,173381,51.457924,-2.589693,95.765799,POINT (-2.58969 51.45792)
286,BS3,357061,169771,51.425314,-2.618968,115.962676,POINT (-2.61897 51.42531)
295,BS4,361147,170721,51.434147,-2.560308,102.504517,POINT (-2.56031 51.43415)
301,BS5,361600,173916,51.462907,-2.554132,74.424220,POINT (-2.55413 51.46291)
302,BS6,362063,174622,51.469287,-2.547538,104.156384,POINT (-2.54754 51.46929)
303,BS7,363099,176769,51.488663,-2.532850,76.470352,POINT (-2.53285 51.48866)
305,BS8,357130,173447,51.458373,-2.618418,116.620348,POINT (-2.61842 51.45837)
306,BS9,356886,176678,51.487400,-2.622330,112.749978,POINT (-2.62233 51.48740)


In [6]:
bristol_innercity.explore('demand', cmap='Reds', tiles='stamen-toner')

In this case, we will assume that we are locating facilities at postcode centroids, and all client demand exists at that centroid. This is obviously an abstraction, but many real-world companies solve their location-allocation problems at exactly this level of abstraction! 
 
To build a `PMedian` object, we use the `PMedian()` class from the `spopt.locate` module: 

In [7]:
model = spopt.locate.PMedian.from_geodataframe(
    gdf_demand=bristol_innercity,
    gdf_fac=bristol_innercity,
    demand_col="geometry", 
    facility_col="geometry",
    weights_cols="demand",
    p_facilities=2
)

This object is a special instance of the `spopt` spatial optimisation library for Python's `locate` class, which supports generic location-allocation problem specifications: 

In [8]:
model

You can see that the underlying `pulp` model is stored in the `.problem` attribute, and it should look similar to the problem we specified above. But, it will be *rather long* because of the number of constraints: 

In [9]:
model.problem

p-median:
MINIMIZE
1.0361187166119716*z_0_1_ + 5.854280146562571*z_0_2_ + 5.369300697314089*z_0_3_ + 4.513015497867477*z_0_4_ + 5.291105242473352*z_0_5_ + 7.513845620280466*z_0_6_ + 3.29945044727688*z_0_7_ + 4.407848489265357*z_0_8_ + 0.8333579535398001*z_1_0_ + 4.196725391446776*z_1_2_ + 3.6199268904725024*z_1_3_ + 3.4387990567545628*z_1_4_ + 4.1810977251195265*z_1_5_ + 6.1885848129723*z_1_6_ + 2.751208623874031*z_1_7_ + 4.2115272011409575*z_1_8_ + 5.7016868224064705*z_2_0_ + 5.081809079349733*z_2_1_ + 6.879057555615924*z_2_3_ + 8.690967861960935*z_2_4_ + 9.726959829138698*z_2_5_ + 12.397384057002862*z_2_6_ + 3.8341406317314197*z_2_7_ + 7.2102067757236545*z_2_8_ + 4.622451429649219*z_3_0_ + 3.8746489764540826*z_3_1_ + 6.080701928157628*z_3_2_ + 3.0152371815273744*z_3_4_ + 3.8324799361605706*z_3_5_ + 6.256921395724978*z_3_6_ + 6.453447951802201*z_3_7_ + 8.379460858327747*z_3_8_ + 2.8209328285944695*z_4_0_ + 2.6724565517274628*z_4_1_ + 5.577816285582547*z_4_2_ + 2.189236951478638*z_4_3_

Even in a "small" problem with eight facilities, we have 91 constraints. 

To solve the model, we use the `.solve()` method (like before in the knapsack problem), but `spopt` requires you to specify the solver you want to use directly. This is because `spopt`'s default problem size is quite large, compared to the default solver that `pulp` encourages you to use: COIN-OR. This ensures that users do not accidentally run *very* long (i.e. many-day) computations that can crash your computer; in order to solve, you have to explicitly specify your solver, preventing accidental solving runs. 

We will use the open-source COIN-OR solver here, `pulp.COIN_CMD()`. This also allows you to provide options to the solver itself, such as `msg=False`, which turns off the verbose printing of model progress: 

In [10]:
model.solve(pulp.COIN_CMD(msg=False))

For the solved `PMedian` object, the `.cli2fac` list records the facility that each client was assigned to in the optimal solution. You can see this below:  

In [11]:
model.cli2fac

[[7], [7], [7], [5], [5], [5], [5], [7], [7]]

The `spopt` package stores this in a list-of-lists form because for *fractional assignment* problems, observation 1 might be assigned to *both* `7` and `5`. For now, though, every client in the `.cli2fac` list should only be assigned to a single facility, and you can see this is the case above. 

In addition, the mapping *in reverse* is stored in the `.fac2cli` list-of-lists: 

In [12]:
model.fac2cli

[[], [], [], [], [], [3, 4, 5, 6], [], [0, 1, 2, 7, 8], []]

Generally, we will want to *stack* the list-of-lists in `.cli2fac` into a single column that corresponds to the row indices in our input client dataframe. I recommend using `numpy.stack()` to do this: 

In [13]:
numpy.stack(model.cli2fac).squeeze()

array([7, 7, 7, 5, 5, 5, 5, 7, 7])

<div class="alert alert-success">
<b>Digression</b>

More experienced Pythonistas may ask why I use `numpy.stack()` instead of something else like `numpy.hstack()`. If you are interested, the reason is that [`numpy.hstack` combines badly with undesirable behavior caused by rounding in the `.solve()` method.](https://github.com/pysal/spopt/issues/364) It is safer to use `numpy.stack()` so that you can identify if there are erroneous allocations arising from these rounding issues.  

</div>

Let's store these allocations to a vector, `allocations`:

In [14]:
allocations = numpy.stack(model.cli2fac)

I find it easier to work with *names*, rather than *locations* in a dataframe, so I will grab the *postcode* for each allocation:

In [15]:
allocation_by_name = bristol_innercity.outward.values[allocations]

Now, to visualize our result, we can add the column back to our input dataframe. Note that `pandas` prefers our input to be a "flat" array, rather than a column vector, so we can use the `.flatten()` method: 

In [16]:
bristol_innercity['allocation_by_name'] = allocation_by_name.flatten()
bristol_innercity.sort_values("allocation_by_name")

,outward,easting,northing,lat,lon,demand,geometry,allocation_by_name
295,BS4,361147,170721,51.434147,-2.560308,102.504517,POINT (-2.56031 51.43415),BS6
301,BS5,361600,173916,51.462907,-2.554132,74.424220,POINT (-2.55413 51.46291),BS6
302,BS6,362063,174622,51.469287,-2.547538,104.156384,POINT (-2.54754 51.46929),BS6
303,BS7,363099,176769,51.488663,-2.532850,76.470352,POINT (-2.53285 51.48866),BS6
264,BS1,358981,174319,51.466347,-2.591879,119.066167,POINT (-2.59188 51.46635),BS8
275,BS2,359125,173381,51.457924,-2.589693,95.765799,POINT (-2.58969 51.45792),BS8
286,BS3,357061,169771,51.425314,-2.618968,115.962676,POINT (-2.61897 51.42531),BS8
305,BS8,357130,173447,51.458373,-2.618418,116.620348,POINT (-2.61842 51.45837),BS8
306,BS9,356886,176678,51.487400,-2.622330,112.749978,POINT (-2.62233 51.48740),BS8


Like before in our knapsack problem, we can see the decision variables that are constructed by `spopt`. They are usually stored directly in the object, ending in `vars`. So, for instance, the facility *location* decision variables, $y_i$, are stored in `.fac_vars`:

In [17]:
model.fac_vars

[y_0_, y_1_, y_2_, y_3_, y_4_, y_5_, y_6_, y_7_, y_8_]

You can work with them directly like we did in the knapsack notebook, using `.varValue` to get the optimal value of that decision variable (for instance): 

In [18]:
bristol_innercity.assign(
    selected_for_depot = [dv.varValue for dv in model.fac_vars]
).sort_values("selected_for_depot")

,outward,easting,northing,lat,lon,demand,geometry,allocation_by_name,selected_for_depot
264,BS1,358981,174319,51.466347,-2.591879,119.066167,POINT (-2.59188 51.46635),BS8,0.0
275,BS2,359125,173381,51.457924,-2.589693,95.765799,POINT (-2.58969 51.45792),BS8,0.0
286,BS3,357061,169771,51.425314,-2.618968,115.962676,POINT (-2.61897 51.42531),BS8,0.0
295,BS4,361147,170721,51.434147,-2.560308,102.504517,POINT (-2.56031 51.43415),BS6,0.0
301,BS5,361600,173916,51.462907,-2.554132,74.424220,POINT (-2.55413 51.46291),BS6,0.0
303,BS7,363099,176769,51.488663,-2.532850,76.470352,POINT (-2.53285 51.48866),BS6,0.0
306,BS9,356886,176678,51.487400,-2.622330,112.749978,POINT (-2.62233 51.48740),BS8,0.0
302,BS6,362063,174622,51.469287,-2.547538,104.156384,POINT (-2.54754 51.46929),BS6,1.0
305,BS8,357130,173447,51.458373,-2.618418,116.620348,POINT (-2.61842 51.45837),BS8,1.0


You can also see the whole matrix of assignment variables, $z_{ij}$, in `cli_assgn_vars`: 

In [19]:
model.cli_assgn_vars

array([[z_0_0_, z_0_1_, z_0_2_, z_0_3_, z_0_4_, z_0_5_, z_0_6_, z_0_7_,
        z_0_8_],
       [z_1_0_, z_1_1_, z_1_2_, z_1_3_, z_1_4_, z_1_5_, z_1_6_, z_1_7_,
        z_1_8_],
       [z_2_0_, z_2_1_, z_2_2_, z_2_3_, z_2_4_, z_2_5_, z_2_6_, z_2_7_,
        z_2_8_],
       [z_3_0_, z_3_1_, z_3_2_, z_3_3_, z_3_4_, z_3_5_, z_3_6_, z_3_7_,
        z_3_8_],
       [z_4_0_, z_4_1_, z_4_2_, z_4_3_, z_4_4_, z_4_5_, z_4_6_, z_4_7_,
        z_4_8_],
       [z_5_0_, z_5_1_, z_5_2_, z_5_3_, z_5_4_, z_5_5_, z_5_6_, z_5_7_,
        z_5_8_],
       [z_6_0_, z_6_1_, z_6_2_, z_6_3_, z_6_4_, z_6_5_, z_6_6_, z_6_7_,
        z_6_8_],
       [z_7_0_, z_7_1_, z_7_2_, z_7_3_, z_7_4_, z_7_5_, z_7_6_, z_7_7_,
        z_7_8_],
       [z_8_0_, z_8_1_, z_8_2_, z_8_3_, z_8_4_, z_8_5_, z_8_6_, z_8_7_,
        z_8_8_]], dtype=object)

If you want to interact with *these* decision variables, you have to loop over the matrix to extract each $z_{ij}$. Generally, this won't be necessary unless you are working with a *fractional assignment* $p$-Median problem, but it is useful to know this regardless: 

In [20]:
n_clients, n_facilities = model.cli_assgn_vars.shape

allocation_matrix = numpy.empty_like(model.cli_assgn_vars)

for client in range(n_clients):
    for facility in range(n_facilities):
        allocation_matrix[client,facility] = model.cli_assgn_vars[client,facility].varValue

In [21]:
allocation_matrix

array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]], dtype=object)

Finally, the distances $d_{ij}$ (sometimes stored altogether in a *cost matrix* $\mathbf{D}$), are contained in the `.aij` attribute: 

In [22]:
model.aij.round(2)

array([[ 0.  ,  1.04,  5.85,  5.37,  4.51,  5.29,  7.51,  3.3 ,  4.41],
       [ 0.83,  0.  ,  4.2 ,  3.62,  3.44,  4.18,  6.19,  2.75,  4.21],
       [ 5.7 ,  5.08,  0.  ,  6.88,  8.69,  9.73, 12.4 ,  3.83,  7.21],
       [ 4.62,  3.87,  6.08,  0.  ,  3.02,  3.83,  6.26,  6.45,  8.38],
       [ 2.82,  2.67,  5.58,  2.19,  0.  ,  0.68,  2.49,  4.8 ,  5.39],
       [ 4.63,  4.55,  8.74,  3.89,  0.96,  0.  ,  2.53,  7.47,  8.02],
       [ 4.83,  4.94,  8.18,  4.67,  2.55,  1.86,  0.  ,  6.94,  6.84],
       [ 3.23,  3.35,  3.86,  7.34,  7.52,  8.36, 10.59,  0.  ,  3.42],
       [ 4.17,  4.96,  7.01,  9.22,  8.17,  8.68, 10.09,  3.3 ,  0.  ]])

By default, `spopt` implements euclidean distance between clients and facilities. But, you can use your own distance metric by passing a `metric` argument (like, `metric='manhattan'`). Alternatively, you can use the much more flexible `PMedian.from_cost_matrix()` function to form a P-Median problem directly from *your own* distance matrix. 

# Do

Now it is time for you to try your hand at solving $p$-Median problems. 

## One again, with feeling

<div class="alert alert-warning">
    
Using the methods we discussed above, can you create a new PMedian problem object to locate eight depots across all Bristol post codes? Are BS6 and BS8 chosen again in this new setup? It may help to make a map using the `.explore()` method. 

</div>

## Keep the old solutions around

Unfortunately, between us discovering the first solution for inner-city postcodes and your new work looking at a *all* of Bristol, the depots in BS6 and BS8 were already built. Thus, we need to re-solve the problem for all of Bristol while *ensuring that* BS6 and BS8 are in the solution. We support this in `spopt` using a `predefined_facilities_col` argument. This column should contain `True` when an observation is *must* be selected for the solution, and `False` when it may is allowed to be omitted from the solution. 

In our case, build the column:

```python
bristol_postcodes['preselected'] = bristol_postcodes.outward.isin(("BS6", "BS8"))
```

and use that as the `predefined_facilities_col` argument in `spopt`.

<div class="alert alert-warning">
    
Using the methods we discussed above, can you create a new PMedian problem object to locate eight depots across all Bristol post codes, so that *BS6* and *BS8* are required to be selected as depots? how do the assigments differ between this case and your previous Bristol-wide solution? It may help to make a map using the `.explore()` method. 

</div>

## Locate depots outside of the city

Land is too expensive in the city. Now, the supervisor wants to make sure that *no* facilities are selected in the city. One way to do this is to use all of the postcodes in *outer Bristol* as facilities, but keep all of the postcodes as demands. To get the postcodes for outer bristol, we invert the selection from inner Bristol:

```python
# in pandas, ~ inverts True/False statements
bristol_outercity = bristol_postcodes[~ is_inner_city].copy()
```

<div class="alert alert-warning">
    
Using the methods we discussed above, can you create a new PMedian problem object to locate eight depots in *outer Bristol* that service *all postcodes in Bristol*? How do the assigments differ between this case and your previous Bristol-wide solution? It may help to make a map using the `.explore()` method. 

</div>

## Challenge: Bristol is far from an isometric plain! 

After a few months using the new depots, you begin to hear about complaints from your drivers about the awkwardness of the depot locations. Specifically, it seems your solver has used Euclidean Distances for everything, which means that you've assumed the Earth is flat, and trucks can drive anywhere in Bristol!  This is a problem for many of your drivers. So, you build a "cost table" that describes the __typical travel time from every postcode to every other postcode__. No doubt this can help!

The code implementing this is rather long and omitted for brevity here, but is available in `01-compute_costs.py`. We will read in the cost table here: 

In [23]:
cost_table = pandas.read_csv("./cost_table.csv")

To see the first few rows: 

In [24]:
cost_table.head()

,origin_postcode,destination_postcode,cost
0,BS10,BS1,1077.4
1,BS10,BS10,0.0
2,BS10,BS11,763.5
3,BS10,BS12,787.4
4,BS10,BS13,955.0


This has a typical "(origin, destination, weight)" structure as is common in storing routing data.

We will use this to provide a bit more realistic costs for the location-allocation decisions. 

To build a cost_matrix from this, you have to use the `.pivot()` method on the `pandas.DataFrame`, like so: 

In [25]:
cost_matrix = cost_table.pivot(index="destination_postcode", columns="origin_postcode", values="cost")
cost_matrix.iloc[0:5, 0:5]

origin_postcode,BS10,BS11,BS12,BS13,BS14
destination_postcode,,,,,
BS1,1077.4,672.8,473.3,286.0,771.4
BS10,0.0,682.8,447.7,652.0,1137.4
BS11,763.5,0.0,588.5,834.4,1164.3
BS12,787.4,619.6,0.0,585.1,1070.5
BS13,955.0,834.3,555.8,0.0,565.1


<div class="alert alert-warning">
    
Using `spopt.locate.PMedian.from_cost_matrix()` function, can you create a new PMedian problem object to locate eight depots in *outer Bristol* that service all postcodes in Bristol _using travel time_ for the cost matrix? How do the assigments differ between this case and your previous Bristol-wide solution? It may help to make a map using the `.explore()` method. 

</div>

## Review

$p$-Median problems are a kind of *spatial optimisation* problem that occurs when trying to locate $p$ facilities and allocate $n$ clients points to these facilities. It requires both *location* decisions (where to place facilities) and *allocation* decisions (what client is served by what facility). As such, is it considered a fundamental *location-allocation* problem, and has structures common to many different kinds of spatial optimisation problems, such as the transportation problem or the warehouse location problem. Many different customisations can be implemented *on top* of the $p$-Median problem, such as the *unweighted*, *continuous*, *capacitated*, or *fractional allocation* problems. Regardless, the $p$-Median problem can be stated and solved easily in Python using the `spopt` library, which wraps the large amount of constraint and variable declarations required to build the $p$-Median model in the underlying `pulp` package. It supports many variants of the $p$-Median problem, including cases where the client sites are not the same as the facility sites and the use of travel time/routing distances.  